In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan

# Функция для расчета робастных стандартных ошибок HC1
def cse(model):
    robust_model = model.get_robustcov_results(cov_type='HC1')
    return robust_model.bse

# Загрузка данных
data = pd.read_csv("Agriculture.csv")

# Построение регрессионной модели
model1 = smf.ols('PRODP ~ FUNG1 + FUNG2 + YDOB1 + YDOB2 + GIRB + INSEC + LABOUR', data=data).fit()

# Вывод результатов с обычными стандартными ошибками
print("Model1 с обычными стандартными ошибками:")
print(model1.summary())

# Создаем модель с робастными ошибками HC1
robust_model = model1.get_robustcov_results(cov_type='HC1')

# Рассчет p-значений для робастных ошибок
t_values = robust_model.params / robust_model.bse
p_values = 2 * stats.t.sf(np.abs(t_values), robust_model.df_resid)

# Создание DataFrame с коэффициентами и ошибками
coef_df = pd.DataFrame({
    'Coefficient': model1.params,
    'Std. Error': model1.bse,
    'p-value': model1.pvalues,
    'Robust Std. Error': robust_model.bse,
    'Robust p-value': p_values
})

# Вывод результатов с робастными ошибками
print("\nModel1 с робастными стандартными ошибками HC1:")
print(coef_df.round(4))

# Проведение теста Бреуша-Пагана
residuals = model1.resid
exog = model1.model.exog
bp_test = het_breuschpagan(residuals, exog)

print("\nТест Бреуша-Пагана:")
print(bp_test[0], bp_test[1])

Model1 с обычными стандартными ошибками:
                            OLS Regression Results                            
Dep. Variable:                  PRODP   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                     111.1
Date:                Mon, 24 Mar 2025   Prob (F-statistic):           5.08e-64
Time:                        02:59:50   Log-Likelihood:                -946.18
No. Observations:                 200   AIC:                             1908.
Df Residuals:                     192   BIC:                             1935.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  